Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

##  Azure ML and IoT Edge Workshop

For more information go to https://aka.ms/aml-tutorial/ai-iot-power-couple


## 1: Specify parameters
Fill in the parameters below.  If you already have IoT Hub or Azure ML workspace, then enter their information here. Otherwise, the parameter names will be used in provisioning new services.

In [26]:
# Pick a subscription ID to provision your services
subscription_id = ""


# Enter Azure region where your services will be provisioned
azure_region=""

# Pick a name for your Azure ML workspace
aml_workspace_name = ""

# Enter the resource group in Azure where your IoT Hub created
resource_group_name = ""

#  Azure IoT Hub name you have created
iot_hub_name=""

# Pick a name for your IoT Edge device
# This is NOT the name of your VM, but it's just a device entry in your IoT Hub, so you can pick any name
iot_device_id=""

In [27]:
# DO NOT CHANGE THIS VALUE for this tutorial
# This is the name of the AML module you deploy to the device
module_name = "machinelearningmodule"

In [ ]:
!az login

In [ ]:
# Load the IoT extension for Azure CLI
!az extension add --name azure-iot

In [28]:
!az account set --subscription $subscription_id

In [29]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## 2: Provision Azure ML Workspace and IoT Hub 
If you already have provisioned these resources, then skip this section and go Section 3.

### 2.1: Provision Azure ML workspace (optional)
After you've done this once you no longer have to run this cell anymore

In [ ]:
ws = Workspace.create(subscription_id = subscription_id,
                resource_group = resource_group_name,
                name = aml_workspace_name,
                location = azure_region)

ws.write_config()

### 2.2: Provision IoT Hub (optional)
If you get an error because there's already one free hub in your subscription, change the SKU to S1. If you get an error that the IoT Hub name isn't available, it means that someone else already has a hub with that name. 


In [ ]:
!az iot hub create --resource-group $resource_group_name --name $iot_hub_name --sku F1

In [ ]:
# Register an IoT Edge device (create a new entry in the Iot Hub)
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $iot_device_id --edge-enabled

## 3: Load resources
Load the Azure ML workspace and get the IoT Edge device connection string from your IoT Hub.

### 3.1 Load the Azure ML workspace.

In [30]:
# Initialize a workspace object from persisted configuration
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


mihdemo
mihhol
eastus
8f0a3a1f-6db3-4c64-917b-e2f5eba6f538


### 3.2: Get the Azure IoT Edge device connection string

In [31]:
# Get the connection string that you will need to enter in the IoT Edge device
!az iot hub device-identity show-connection-string --device-id $iot_device_id --hub-name $iot_hub_name

This command has been deprecated and will be removed in a future release. Use 'az iot hub device-identity connection-string show' instead.
{
  "connectionString": "HostName=a9dmhub.azure-devices.net;DeviceId=edgevm;SharedAccessKey=q43BF1At+/bzDlVke9r7hlIngaz17C/NSCZGh3eSdy8="
}


## 4: Anomaly Detection Model

## 4.1 Train the model

In [32]:
!pip install pandas
!pip install sklearn

In [33]:
# Load the data set
import pandas
import numpy
import pickle


from sklearn import tree
from sklearn.model_selection import train_test_split

In [34]:
temp_data = pandas.read_csv('temperature_data.csv')
temp_data

,machine_temperature,machine_pressure,ambient_temperature,ambient_humidity,anomaly
0,21.018765,1.002138,20.768628,24,0
1,21.965850,1.110034,21.223653,26,0
2,22.999720,1.227816,20.872120,25,0
3,23.731603,1.311195,21.470454,25,0
4,24.902941,1.444639,20.895378,24,0
...,...,...,...,...,...
613,105.877218,10.669556,20.533100,25,1
614,105.990477,10.682459,21.383384,24,1
615,105.642759,10.642846,21.347985,24,1
616,105.742701,10.654232,20.568153,26,1


In [35]:
# Load features and labels
X, Y = temp_data[['machine_temperature', 'machine_pressure', 'ambient_temperature', 'ambient_humidity']].values, temp_data['anomaly'].values

In [36]:
# Split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

# Change regularization rate and you will likely get a different accuracy.
reg = 0.01

# Train a decision tree on the training set
#clf1 = LogisticRegression(C=1/reg).fit(X_train, Y_train)
clf1 = tree.DecisionTreeClassifier()
clf1 = clf1.fit(X_train, Y_train)
print (clf1)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [37]:
# Evaluate the test set
accuracy = clf1.score(X_test, Y_test)

print ("Accuracy is {}".format(accuracy))

Accuracy is 0.9907834101382489


In [38]:
# Serialize the model and write to disk
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()
print ("Exported the model to model.pkl")

Exported the model to model.pkl


In [39]:
# Test the model by importing it and providing a sample data point
print("Import the model from model.pkl")
f2 = open('model.pkl', 'rb')
clf2 = pickle.load(f2)

# Normal (not an anomaly)
#X_new = [[24.90294136, 1.44463889, 20.89537849, 24]]
#X_new = [[33.40859853, 2.413637808, 20.89162813, 26]]
#X_new = [[34.42109181, 2.528985143, 21.23903786, 25]]

# Anomaly
X_new = [[33.66995566, 2.44341267, 21.39450979, 26]]
#X_new = [[105.5457931, 10.63179922, 20.62029994, 26]]

print ('New sample: {}'.format(X_new))

pred = clf2.predict(X_new)
print('Predicted class is {}'.format(pred))

Import the model from model.pkl
New sample: [[33.66995566, 2.44341267, 21.39450979, 26]]
Predicted class is [1]


### 4.2 Register Model

You can add tags and descriptions to your models. Note you need to have a `model.pkl` file in the current directory. The below call registers that file as a model with the same name `model.pkl` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric.

In [40]:
from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "anomaly", 'type': "classification"},
                       description = "Sample anomaly detection model for IOT tutorial",
                       workspace = ws)

Registering model model.pkl


In [41]:
print(model.name, model.description, model.version, sep = '\t')

model.pkl	Sample anomaly detection model for IOT tutorial	2


### 4.2 Create Docker Image

Create the driver file.  Note that the `model.pkl` file in the get_model_path call is referring to a model named `model.pkl` registered under the workspace (what you registered above). It is NOT referencing the local file.

In [42]:
%%writefile iot_score.py
# This script generates the scoring file
# with the init and run functions needed to 
# operationalize the anomaly detection sample

import pickle
import json
import pandas
import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(input_str):
    try:
        input_json = json.loads(input_str)
        input_df = pandas.DataFrame([[input_json['machine']['temperature'],input_json['machine']['pressure'],input_json['ambient']['temperature'],input_json['ambient']['humidity']]])
        pred = model.predict(input_df)
        print("Prediction is ", pred[0])
    except Exception as e:
        result = str(e)
        
    if pred[0] == 1:
        input_json['anomaly']=True
    else:
        input_json['anomaly']=False
        
    return [json.dumps(input_json)]


Overwriting iot_score.py


#### 4.2.1 Using New Azure ML environment and DEFAULT_CPU_IMAGE base docker image to setup inference env

In [43]:
# Set the web service configuration (using default here)
from azureml.core.model import InferenceConfig

# from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment, DEFAULT_CPU_IMAGE

env = Environment("deploytoedgeenv")
# Please see [Azure ML Containers repository](https://github.com/Azure/AzureML-Containers#featured-tags)
# for open-sourced GPU base images.
env.docker.base_image = DEFAULT_CPU_IMAGE
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "scikit-learn","numpy"],
    pip_packages=["azureml-defaults"],
)

inference_config = InferenceConfig(
    entry_script="iot_score.py", environment=env
)

In [44]:
# provide name of azure contaienr image and tag
imagename = "anomalydetect-skl"
imagelabel = "0.1"

#### 4.2.2 Replace the deprecated Azure ML SDK Image.create() with New Model.package()

In [45]:
# Builds an image in ACR.

package = Model.package(
    ws,
    [model],
    inference_config=inference_config,
    image_name=imagename,
    image_label=imagelabel,
)
package.wait_for_creation(show_output=True)

print("ACR:", package.get_container_registry)
print("Image:", package.location)

2021/02/20 13:45:19 Downloading source code...
2021/02/20 13:45:20 Finished downloading source code
2021/02/20 13:45:21 Creating Docker network: acb_default_network, driver: 'bridge'
2021/02/20 13:45:21 Successfully set up Docker network: acb_default_network
2021/02/20 13:45:21 Setting up Docker configuration...
2021/02/20 13:45:22 Successfully set up Docker configuration
2021/02/20 13:45:22 Logging in to registry: mihdemo.azurecr.io
2021/02/20 13:45:23 Successfully logged into mihdemo.azurecr.io
2021/02/20 13:45:23 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/02/20 13:45:23 Scanning for dependencies...
2021/02/20 13:45:24 Successfully scanned dependencies
2021/02/20 13:45:24 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1@sha256:81ca5cdfeefc2a25afff41e66151e97b8a36c0a43d0cd0056af26f01dd37885e
sha256

# This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

You can add tags and descriptions to images. Also, an image can contain multiple models.

from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="iot_score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "iot", 'type': "classification"},
                                 description = "IOT Edge anomaly detection demo")


image = Image.create(name = "tempanomalydetection",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

Note that following command can take few minutes. 

image.wait_for_creation(show_output = True)

List images by tag and find out the detailed build log for debugging.

for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

## 5: Test model on Azure Container Instance
You can test the performance of your model by deploying the container to ACI.  Note that the service creation can take few minutes.

In [51]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "iot", 'type': "classification"}, 
                                               description = 'IOT Edge anomaly detection demo')

### 5.1 Using new Azure ML SDK - Model.deploy() to create image and deploy to ACI as service

In [52]:
aci_service_name = 'my-aci-service-anomalydetection'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service my-aci-service-anomalydetection
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


from azureml.core.webservice import Webservice

aci_service_name = 'tempsensor-iotedge-ml-1'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

### 5.2: Test web service

Call the web service with some dummy input data to get a prediction.

In [53]:
import json

# Anomaly
#test_sample = json.dumps({ "machine": { "temperature": 33.66995566, "pressure": 2.44341267 }, \
#                          "ambient": { "temperature": 21.39450979, "humidity": 26 },\
#                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

# Normal
test_sample = json.dumps({ "machine": { "temperature": 31.16469009, "pressure": 2.158002669 }, \
                          "ambient": { "temperature": 21.17794693, "humidity": 25 },\
                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aci_service.run(input_data = test_sample)
print(prediction)

['{"machine": {"temperature": 31.16469009, "pressure": 2.158002669}, "ambient": {"temperature": 21.17794693, "humidity": 25}, "timeCreated": "2017-10-27T18:14:02.4911177Z", "anomaly": false}']


### 5.3: Delete ACI to clean up

In [ ]:
aci_service.delete()

## 6: Deploy container to Azure IoT Edge device
Create a deployment.json file that contains the modules you want to deploy to the device and the routes.  Then push this file to the IoT Hub, which will then send it to the IoT Edge device.  The IoT Edge agent will then pull the Docker images and run them.

This section pulls the container registry information from your workspace. Machine Learning workspaces don't automatically provision container registries when they're created, but wait until there's a container image to store. In the case of this tutorial, the container image was created in section 4.2. So in this section, we update the workspace config information. Your container registry still may not be ready by the time you get to this step. Keep running the first cell of this section until it finds the registry information. You can also check to see if the container registry exists yet by viewing the workspace overview page in the Azure portal.

In [48]:
acr_name = package.location.split("/")[0]
reg_name = acr_name.split(".")[0]
subscription_id = ws.subscription_id

print("{}".format(acr_name))
print("{}".format(subscription_id))

# TODO: Derive image_location through code.
image_location = acr_name + "/" + imagename + ":" + imagelabel

print("{}".format(image_location))

# Fetch username, password of ACR.
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry

client = ContainerRegistryManagementClient(ws._auth, subscription_id)
result = client.registries.list_credentials(
    ws.resource_group, reg_name, custom_headers=None, raw=False
)

username = result.username
password = result.passwords[0].value

mihdemo.azurecr.io
8f0a3a1f-6db3-4c64-917b-e2f5eba6f538
mihdemo.azurecr.io/anomalydetect-skl:0.1


# Update the workspace object
ws = Workspace.from_config()

# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

In [49]:
file = open('iot-workshop-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [50]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

[
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "LIZcMzyZtNuKHdVWSEggp9X1bSuQQCpQgD8lkGHNCow=",
        "secondaryKey": "CW3JZ9WZuTYX6a6TdIy56EvLBJgVFI89Ra88FF1Ugt8="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Disconnected",
    "connectionStateUpdatedTime": "0001-01-01T00:00:00+00:00",
    "deviceId": "edgevm",
    "etag": "MzQ2NTg0NzA4",
    "generationId": "637479303933244018",
    "lastActivityTime": "0001-01-01T00:00:00+00:00",
    "managedBy": null,
    "moduleId": "$edgeAgent"
  },
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "0OxAz6tZfU4+SffUu4PqkvDTtyCaogNaBIGWMJBFY50=",
        "secondaryKey": "zKzeubXtbfLmSGi2T8JQh4iZlOam9mbLNgv1k91hkhk="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null

## Congratulations!
You made it to the end of the tutorial!  You can monitor messages from your edge device to your IoT Hub with VS Code and the [Azure IoT Hub Toolkit](https://marketplace.visualstudio.com/items?itemName=vsciot-vscode.azure-iot-toolkit) extension.